In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense

In [3]:
def read_table(path):
    """Reads table and drops irrelevant columns and rows.
    Returns an analysis ready DataFrame"""
    df = pd.read_table('EMG/'+path)
    df.drop(columns=['time'],inplace=True)
    df = df[df['class']!=0]
    return df

In [4]:
df = read_table('1/2_1.txt')
for n in range(2,37):
    path = f'{n}/2_{n}.txt'
    df = pd.concat([df, read_table(path)])

In [5]:
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class
816,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
817,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
818,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
819,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
820,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
...,...,...,...,...,...,...,...,...,...
48664,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
48665,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
48666,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
48667,0.00014,0.00003,-0.00001,0.00001,-0.00012,0.00006,-0.00007,-0.00013,6


In [6]:
df.reset_index(drop=True,inplace=True)

In [7]:
df['class'].value_counts()

class
4    127509
6    127490
5    125881
3    124058
2    121604
1    121539
7      6609
Name: count, dtype: int64

In [8]:
mapping = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6}
df['class'] = df['class'].map(mapping)

In [9]:
df['class'].value_counts()

class
3    127509
5    127490
4    125881
2    124058
1    121604
0    121539
6      6609
Name: count, dtype: int64

In [10]:
X = df.drop(columns=['class'])
y = df['class']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In [12]:
X_train

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
407254,-0.00002,-0.00006,-0.00009,-0.00007,-0.00005,0.00000,-0.00002,0.00000
217942,-0.00019,-0.00003,-0.00043,-0.00128,0.00071,0.00027,-0.00025,0.00002
322344,-0.00005,-0.00012,-0.00012,-0.00020,-0.00003,-0.00071,-0.00019,-0.00011
94715,-0.00002,-0.00004,-0.00002,-0.00003,-0.00001,0.00002,-0.00013,0.00009
118478,0.00001,-0.00002,-0.00002,0.00001,0.00009,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...
594986,-0.00002,0.00000,0.00002,0.00001,-0.00001,-0.00002,-0.00001,0.00000
133674,-0.00014,-0.00005,-0.00021,0.00011,0.00011,0.00018,0.00016,0.00089
671370,-0.00006,-0.00005,-0.00009,-0.00007,-0.00035,0.00001,0.00022,-0.00009
106050,-0.00001,-0.00006,-0.00009,-0.00007,0.00009,-0.00001,-0.00006,0.00004


In [13]:
def scaler(df):
    """Normalizes the data, returns a DataFrame with the new scaled numbers"""
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    df_scaled = pd.DataFrame(scaled, columns=df.columns)
    return df_scaled

In [14]:
X_train = scaler(X_train)
X_test = scaler(X_test)

In [26]:
model = Sequential()
model.add(Dense(10, input_dim=8, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(7,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 10)                90        
                                                                 
 dense_10 (Dense)            (None, 10)                110       
                                                                 
 dense_11 (Dense)            (None, 7)                 77        
                                                                 
Total params: 277
Trainable params: 277
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=50)

Epoch 1/20
12076/12076 [==============================] - 7s 544us/step - loss: 1.1392 - accuracy: 0.5629
Epoch 2/20
12076/12076 [==============================] - 6s 511us/step - loss: 1.0348 - accuracy: 0.6065
Epoch 3/20
12076/12076 [==============================] - 6s 513us/step - loss: 1.0197 - accuracy: 0.6116
Epoch 4/20
12076/12076 [==============================] - 6s 521us/step - loss: 1.0122 - accuracy: 0.6140
Epoch 5/20
12076/12076 [==============================] - 7s 561us/step - loss: 1.0070 - accuracy: 0.6154
Epoch 6/20
12076/12076 [==============================] - 6s 500us/step - loss: 1.0026 - accuracy: 0.6153
Epoch 7/20
12076/12076 [==============================] - 7s 561us/step - loss: 0.9964 - accuracy: 0.6151
Epoch 8/20
12076/12076 [==============================] - 6s 510us/step - loss: 0.9923 - accuracy: 0.6143
Epoch 9/20
12076/12076 [==============================] - 6s 515us/step - loss: 0.9908 - accuracy: 0.6141
Epoch 10/20
12076/12076 [=====================

In [17]:
model.evaluate(X_test,y_test)

4717/4717 [==============================] - 3s 528us/step - loss: 0.9714 - accuracy: 0.6245


[0.971442461013794, 0.6245412230491638]

In [ ]:
df = read_table('1/2_1.txt')
for n in range(2,37):
    path = f'{n}/2_{n}.txt'
    df = pd.concat([df, read_table(path)])